# Notebook for midterm project - Tudor

### Importing the libraries and connection to database

In [24]:
#import libraries
import requests
import json
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

pd.set_option('display.max_columns', None)


In [25]:
#connect to database and get data
#host
host = 'mid-term-project.ca2jkepgjpne.us-east-2.rds.amazonaws.com'
#port
port = '5432'
#user
user = 'lhl_student'
#password
password = 'lhl_student'
#database
database = 'mid_term_project'
#connect to database
conn = 'postgresql://{}:{}@{}:{}/{}'.format(user, password, host, port, database)

## Data Exploration

### Table information

In [3]:
#pull the list of tables from the database
tables = pd.read_sql_query('SELECT * FROM information_schema.tables WHERE table_schema = \'public\'', conn)

In [4]:
tables

,table_catalog,table_schema,table_name,table_type,self_referencing_column_name,reference_generation,user_defined_type_catalog,user_defined_type_schema,user_defined_type_name,is_insertable_into,is_typed,commit_action
0,mid_term_project,public,flights_test,BASE TABLE,None,None,None,None,None,YES,NO,None
1,mid_term_project,public,flights,BASE TABLE,None,None,None,None,None,YES,NO,None
2,mid_term_project,public,passengers,BASE TABLE,None,None,None,None,None,YES,NO,None
3,mid_term_project,public,fuel_comsumption,BASE TABLE,None,None,None,None,None,YES,NO,None


### Columns

In [5]:
#get column names from the table
columns = pd.read_sql_query('SELECT * FROM information_schema.columns WHERE table_schema = \'public\'', conn)

In [6]:
#coloumns to csv
columns.to_csv('columns.csv')

### Data selection

In [7]:
#select <0.3% random rows from the flights table
flights_ran_subset = pd.read_sql_query('SELECT * FROM flights WHERE random() < 0.003', conn)
flights_ran_subset.to_csv('flights_ran_subset.csv')

### Some checks on the data

In [8]:
#how many rows are we working with
pd.read_sql_query('SELECT COUNT(*) FROM flights', conn)
#15927485 rows

#lets see how many rows we have in the flight_test table
pd.read_sql_query('SELECT COUNT(*) FROM flights_test', conn)
#660556 rows


#count the number of rows by origin city
pd.read_sql_query('SELECT origin_city_name, COUNT(*) FROM flights GROUP BY origin_city_name', conn).to_csv('origin_city_name.csv')
#all flights leave USA, the Ontario,CA had be going for a sec there
pd.read_sql_query('SELECT dest_city_name, COUNT(*) FROM flights GROUP BY dest_city_name', conn).to_csv('dest_city_name.csv')
#and land in USA

In [9]:
#count the number of rows bewtween fl_date 2018-12-01 and 2019-01-01 (inclusive?, need to check)
pd.read_sql_query("SELECT COUNT(*) FROM flights WHERE fl_date between '2018-12-01' and '2019-01-01'", conn)

,count
0,669008


In [10]:
#count the number of rows bewtween fl_date 2019-12-01 and 2020-01-01 (inclusive?, need to check)
pd.read_sql_query("SELECT COUNT(*) FROM flights WHERE fl_date between '2019-12-01' and '2020-01-01'", conn)

,count
0,679941


In [11]:
#lets see how many different flights are in the flight table
pd.read_sql_query("SELECT op_carrier_fl_num FROM flights WHERE fl_date between '2018-12-01' and '2018-12-31' GROUP BY op_carrier_fl_num", conn)

,op_carrier_fl_num
0,1
1,2
2,3
3,4
4,5
...,...
7012,9378
7013,9379
7014,9380
7015,9381


In [12]:
#lets see how many different flights we are working with in december 2018
pd.read_sql_query("SELECT op_carrier_fl_num FROM flights WHERE fl_date between '2018-12-01' and '2018-12-31' GROUP BY op_carrier_fl_num", conn)

,op_carrier_fl_num
0,1
1,2
2,3
3,4
4,5
...,...
7012,9378
7013,9379
7014,9380
7015,9381


In [13]:
#check number of rows with arr_delay > 0
pd.read_sql_query("SELECT COUNT(*) FROM flights WHERE arr_delay > 0", conn)

,count
0,5475609


In [14]:
#percent of rows with arr_delay > 0
5475609 / 15927485 * 100

34.378365448154554

## Feature Engineering

### airport_id based features

#### Rank of Percent of flights delayed

In [16]:
#count of flights with arr_delay > 0
delay = pd.read_sql_query("SELECT origin_airport_id, count(*) FROM flights WHERE dep_delay > 0 GROUP BY origin_airport_id", conn)

#total flights
total = pd.read_sql_query("SELECT origin_airport_id, count(*) FROM flights GROUP BY origin_airport_id", conn)

#merge df into one, then add a column with the percentage of delay
delay_percent = delay.merge(total, on='origin_airport_id')
delay_percent['delay_percent'] = delay_percent['count_x'] / delay_percent['count_y'] * 100

#create new feature table for origin with delay_percent number
airport_features = delay_percent[['origin_airport_id', 'delay_percent']]

#sort the table by delay_percent
airport_features = airport_features.sort_values(by='delay_percent', ascending=True)

#create a new column with the rank of the delay_percent
airport_features['Ranking'] = range(1, len(airport_features) + 1)

#create CSV file
airport_features.to_csv('airport_features.csv')

### Tail number features

#### Age of aircraft

In [66]:
#import csv with date built
aircraft = pd.read_csv('aircraftDatabase.csv', low_memory=False)

#leave only registration and built year
aircraft = aircraft[['registration', 'built']]

#rename registration to tail_num
aircraft = aircraft.rename(columns={'registration': 'tail_num'})

In [67]:
#clean aircraft table
print(aircraft.shape)
aircraft = aircraft.dropna()
print(aircraft.shape)
aircraft = aircraft.drop_duplicates()
print(aircraft.shape)

(460000, 2)
(244135, 2)
(244127, 2)


In [68]:
#make table of aircraft from flights table
tail_num_features = pd.read_sql_query("SELECT DISTINCT tail_num, op_unique_carrier FROM flights", conn)

#left join aircraft table to tail_num_features
tail_num_features = pd.merge(tail_num_features, aircraft, how='left', on='tail_num')

#drop na values based on tail_num_features.tail_num.isna() AKA no tail_num
tail_num_features = tail_num_features[~tail_num_features.tail_num.isna()]

#change built to first 4 digits to signify year
tail_num_features['built'] = tail_num_features['built'].str[:4]

In [69]:
#create new df with op_unique_carrier and built
carrier_built_avg = tail_num_features[['op_unique_carrier', 'built']]

#drop na's
carrier_built_avg = carrier_built_avg.dropna()

#change built to int
carrier_built_avg['built'] = carrier_built_avg['built'].astype(int)

#create new df with average built year for each op_unique_carrier
carrier_built_avg = carrier_built_avg.groupby('op_unique_carrier').agg({'built': 'mean'})
#round built to nearest year
carrier_built_avg['built'] = carrier_built_avg['built'].round()

#change back to str
carrier_built_avg['built'] = carrier_built_avg['built'].astype(str)

#take only first 4 digits
carrier_built_avg['built'] = carrier_built_avg['built'].str[:4]

In [70]:
#fill in na's from tail_num_features['built'] with carrier_built_avg['built']

#merge tail_num_features and carrier_built_avg
tail_num_features = tail_num_features.merge(carrier_built_avg, on='op_unique_carrier')

#create new build column
tail_num_features['build'] = tail_num_features['built_x'].where(tail_num_features['built_x'].notnull(), tail_num_features['built_y'])

#drop op_unique_carrier, built_x and built_y
tail_num_features = tail_num_features.drop(['op_unique_carrier', 'built_x', 'built_y'], axis=1)

#drop duplicates
tail_num_features = tail_num_features.drop_duplicates()

#create new column with age of aircraft
tail_num_features['age'] = 2020 - tail_num_features['build'].astype(int)

#reindex the table based on tail_num
tail_num_features = tail_num_features.set_index('tail_num')

#export to csv
tail_num_features.to_csv('tail_num_features.csv')

### Flight Number features (grouped by op_carrier_fl_num AND rounded crs_dep_time)

#### Create a table of unique op_carrier_fl_num and rounded crs_dep_time to serve as a feature table

In [27]:
#make a function to clean up the rounded crs_dep_time
def clean_round(df):
    ### 
    # change the round column in a dataframe to the first 2 digits signifying the hour
    ###
    df['round'] = df['round'].astype(str)
    df['round'] = df['round'].str.zfill(6)
    df['round'] = df['round'].str[:2]
    return df


In [28]:
#create a table for all flights not just delayed
flight_num_features = pd.read_sql_query("SELECT op_carrier_fl_num, round(crs_dep_time, -2) as round, AVG(carrier_delay + weather_delay + nas_delay + security_delay + late_aircraft_delay) as Avg_sum_of_delays, AVG(arr_delay) as avg_del_total, count(*) as total_flights FROM flights GROUP BY op_carrier_fl_num, round", conn)

#clean round
clean_round(flight_num_features)

#reindex the table based on op_carrier_fl_num AND round
flight_num_features = flight_num_features.set_index(['op_carrier_fl_num', 'round'])

#### 'avg_sum_of_delays', 'avg_del_total', 'total_flights','avg_del_if_delayed', 'times_delayed', 'percent_delayed'

In [29]:
#find average delay for each group of op_carrier_fl_num AND crs_dep_time_rounded
#filter in rows with arr_delay > 0
flight_num_sub_if_delayed = pd.read_sql_query("SELECT op_carrier_fl_num, round(crs_dep_time, -2) as round, AVG(arr_delay) as avg_del_if_delayed, count(arr_delay) as times_delayed FROM flights WHERE arr_delay > 0 GROUP BY op_carrier_fl_num, round", conn)

#clean round
clean_round(flight_num_sub_if_delayed)

#reindex the table based on op_carrier_fl_num AND round
flight_num_sub_if_delayed = flight_num_sub_if_delayed.set_index(['op_carrier_fl_num', 'round'])

#join flight_num_features and flight_num_sub_if_delayed
flight_num_features = pd.merge(flight_num_features, flight_num_sub_if_delayed, how='left', left_index=True, right_index=True)

#drop duplicate rows
flight_num_features = flight_num_features.drop_duplicates()

#fill in NA's with 0
flight_num_features = flight_num_features.fillna(0)

#change times_delayed to int
flight_num_features['times_delayed'] = flight_num_features['times_delayed'].astype(int)

#create new column with the percentage of flights that were delayed
flight_num_features['percent_delayed'] = (flight_num_features['times_delayed'] / flight_num_features['total_flights'] * 100).round(2)

#export to csv
flight_num_features.to_csv('flight_num_features.csv')

### Carrier features ###

#### 'avg_delay', 'avg_carrier_delay', 'avg_delay_if_del', 'avg_carrier_delay_if_del', 'total_flights', 'delayed_flights', 'percent_delayed'

In [32]:
#create a table of unique op_unique_carrier from flights
carrier_features = pd.read_sql_query("SELECT op_unique_carrier, AVG(arr_delay) as avg_delay, AVG(carrier_delay) as avg_carrier_delay, count(*) as total_flights FROM flights GROUP BY op_unique_carrier", conn)

#reindex the table based on op_unique_carrier
carrier_features = carrier_features.set_index('op_unique_carrier')

#create a table for delayed flights
carrier_features_del = pd.read_sql_query("SELECT op_unique_carrier, AVG(arr_delay) as avg_delay_if_del, AVG(carrier_delay) as avg_carrier_delay_if_del, count(*) as delayed_flights FROM flights WHERE arr_delay > 0 GROUP BY op_unique_carrier", conn)

#reindex the table based on op_unique_carrier
carrier_features_del = carrier_features_del.set_index('op_unique_carrier')

#join carrier_features and carrier_features_del
carrier_features = pd.merge(carrier_features, carrier_features_del, how='left', left_index=True, right_index=True)

#create new column for percent of flights that were delayed
carrier_features['percent_delayed'] = (carrier_features['delayed_flights'] / carrier_features['total_flights'] * 100).round(2)

carrier_features.to_csv('carrier_features.csv')